In [1]:
import pandas as pd
import plotly.graph_objects as go
from signals import *

signal_file = "./telegram/2020-21-01_TelegramDump.json"

In [2]:
def analyse_signal(df, signal):
    """
    Process signal using SL and TP.
    TODO: accound for spread and fees
    TODO: Warn if slippage is high
    """
    real_entry = signal.entry_price
    
    for dt, ohlc in df.loc[signal.datetime_str:].iterrows():
        if signal.type == "BUY":
            if ohlc.High > signal.take_profit:
                return signal.take_profit - real_entry

            if ohlc.Low < signal.stop_loss:
                return signal.stop_loss - real_entry
            
        if signal.type == "SELL":
            if ohlc.Low < signal.take_profit:
                return real_entry - signal.take_profit

            if ohlc.High > signal.stop_loss:
                return real_entry - signal.stop_loss
        
    return 0.0
        

In [3]:
beta_tp1 = Signal(ticker="AUDCAD", datetime_str="2020-01-14 13:19", 
                  entry_price=0.9020, stop_loss=0.9075, take_profit=0.9000)
df = load_ticker(beta_tp1.ticker)
print("TP1", analyse_signal(df, beta_tp1) * pip_factor(beta_tp1.ticker))

TP1 20.000000000000018


In [ ]:
# TP1 Analysis using trade signals as given
signals = load_beta_trades(signal_file)
total_profit = 0.0
wins = 0
losses = 0
for signal in signals["TP3"]:
    df = load_ticker(signal.ticker, 60)
    #signal.set_stop_loss_pips(24)
    #signal.set_take_profit_pips(70)
    
    profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
    if profit > 0:
        wins += 1
    else:
        losses += 1
    total_profit += profit
    print(signal, profit)
print("Default:", total_profit, wins, losses)

In [ ]:
# Given TP Stop loss analysis
signals = load_beta_trades(signal_file)

stops = range(1, 150, 1)
profits = []
take_profit = "TP3"

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals[take_profit]:
        signal.set_stop_loss_pips(stop)
        df = load_ticker(signal.ticker, 60)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    #print(stop, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title=f"BETA: Stop Loss Optimisation for {take_profit}",
    xaxis_title="Stop Loss (pips)",
    yaxis_title="Profit (pips)",
    title_x=0.5
)
fig.show()


In [11]:
# Take Profit analysis
signals = load_beta_trades(signal_file)

stop_loss = 13
take_profits = range(10, 100, 1)
profits = []

for take_profit in take_profits:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals:
        #signal.set_stop_loss_pips(stop_loss)
        signal.set_take_profit_pips(take_profit)
        df = load_ticker(signal.ticker, 60)
        if df is None:
            continue
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(take_profit, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(take_profits), y=profits))
fig.update_layout(
    title=f"BETA: Take Profit Optimisation (Stop Loss = {stop_loss} pips)",
    title_x=0.5,
    xaxis_title="Take Profit (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()
    

10 1206.099999999931 331 31
11 1285.1000000003037 328 34
12 1195.1000000003405 323 39
13 1276.0999999999472 319 43
14 1309.099999999984 315 47
15 1333.100000000033 311 51
16 1644.099999999975 311 51
17 1727.1000000000372 308 54
18 1687.5000000000673 305 57
19 1812.500000000026 303 59
20 2059.4999999999973 302 60
21 2088.4999999999627 300 62
22 2085.4999999997217 297 65
23 2339.4999999999927 296 66
24 2386.500000000005 294 68
25 2475.499999999992 291 71
26 2476.5000000000277 288 74
27 2431.500000000041 285 77
28 2535.5000000000286 283 79
29 2547.500000000074 280 82
30 2570.499999999753 277 85
31 2532.7000000000053 275 87
32 2675.699999999959 274 88
33 2753.700000000007 272 90
34 2931.700000000048 271 91
35 2858.6999999999516 268 94
36 2465.7000000000403 262 100
37 2660.700000000082 261 101
38 2713.6999999999903 259 103
39 2642.7000000000294 256 106
40 2718.7000000000635 254 108
41 2579.6999999999844 251 111
42 2830.700000000019 251 111
43 2968.700000000063 250 112
44 2928.8000000000234 

In [8]:
# Stop Loss analysis
signals = load_beta_trades(signal_file)

take_profit = 52
stops = range(10, 100, 1)
profits = []

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals:
        signal.set_stop_loss_pips(stop)
        signal.set_take_profit_pips(take_profit)
        df = load_ticker(signal.ticker, 60)
        if df is None:
            continue
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(stop, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title=f"BETA: Stop Loss Optimisation (Take Profit = {take_profit} pips)",
    title_x=0.5,
    xaxis_title="Stop Loss Level (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()
    

10 2766.0000000001164 103 259
11 3010.9999999998577 111 251
12 3271.999999999852 119 243
13 3288.999999999959 123 239
14 3247.9999999999686 126 236
15 3212.999999999942 129 233
16 2979.9999999999905 129 233
17 2815.9999999999573 130 232
18 2723.999999999956 132 230
19 2778.0000000000127 136 226
20 2840.0000000000496 140 222
21 2764.000000000092 142 220
22 2544.000000000083 142 220
23 2623.999999999993 146 216
24 2636.0000000000205 149 213
25 2423.0000000000405 149 213
26 2522.000000000014 153 209
27 2550.0000000000405 156 206
28 2824.000000000059 162 200
29 2948.000000000057 166 196
30 3162.0000000000996 171 191
31 3136.999999999959 173 189
32 3032.0000000000064 174 188
33 3079.9999999999977 176 186
34 2895.9999999999814 176 186
35 2799.0000000000696 177 185
36 2880.000000000015 180 182
37 2967.0000000000095 183 179
38 3150.0000000000823 187 175
39 3158.999999999982 189 173
40 3079.9999999999673 190 172
41 3096.0000000000305 192 170
42 3304.0000000000236 196 166
43 3330.000000000018 19

In [ ]:
# Example plotly candlestick plot

dfp = df.loc["2019-01-14 21:00":"2019-01-14 22:00"]
fig = go.Figure(data=[go.Candlestick(x=dfp.index,
                open=dfp['Open'],
                high=dfp['High'],
                low=dfp['Low'],
                close=dfp['Close'])])

fig.show()